In [41]:
import pandas as pd
import numpy as np

In [42]:
from cbp.entity.artifact_entity import DataIngestionArtifact

In [43]:
import os
ROOT_DIR = os.getcwd()
print(ROOT_DIR)

/config/workspace


In [44]:
from cbp.component.data_ingestion import DataIngestion
from cbp.config.configuration import Configuration

In [45]:
data_ingestion = Configuration(ROOT_DIR)
data_ingestion_config = data_ingestion.get_data_ingestion_config()

In [46]:
print(data_ingestion_config)

DataIngestionConfig(dataset_download_url='https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz', tgz_download_dir='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/tgz_data', raw_data_dir='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/raw_data', ingested_train_dir='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/ingested_data/train', ingested_test_dir='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/ingested_data/test')


In [47]:
data_ingestion = DataIngestion(data_ingestion_config)

In [48]:
data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

HousingException: 
        Error occured in script: 
        [ /config/workspace/cbp/component/data_ingestion.py ] at 
        try block line number: [116] and exception block line number: [120] 
        error message: [
        Error occured in script: 
        [ /config/workspace/cbp/component/data_ingestion.py ] at 
        try block line number: [29] and exception block line number: [41] 
        error message: [[Errno 21] Is a directory: '/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/tgz_data']
        ]
        

In [ ]:
data_ingestion_artifact

DataIngestionArtifact(train_file_path='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/ingested_data/train/housing.csv', test_file_path='/config/workspace/cbp/artifact/data_ingestion/2022-12-03-22-34-12/ingested_data/test/housing.csv', is_ingested=True, message='Data ingestion completed successfully.')

In [ ]:
train_data  = pd.read_csv(data_ingestion_artifact.train_file_path)

In [ ]:
test_data  = pd.read_csv(data_ingestion_artifact.test_file_path)

In [ ]:
from cbp.util.util import  read_yaml_file
configuration = Configuration(ROOT_DIR)
data_validation_config = configuration.get_data_validation_config(ROOT_DIR)

In [ ]:
schema_file_obj = read_yaml_file(data_validation_config.schema_file_path)

In [ ]:
schema_file_obj

{'columns': {'longitude': 'float',
  'latitude': 'float',
  'housing_median_age': 'float',
  'total_rooms': 'float',
  'total_bedrooms': 'float',
  'population': 'float',
  'households': 'float',
  'median_income': 'float',
  'median_house_value': 'float',
  'ocean_proximity': 'category'},
 'numerical_columns': ['longitude',
  'latitude',
  'housing_median_age',
  'total_rooms',
  'total_bedrooms',
  'population',
  'households',
  'median_income'],
 'categorical_columns': ['ocean_proximity'],
 'target_column': 'median_house_value',
 'domain_value': {'ocean_proximity': ['<1H OCEAN',
   'INLAND',
   'ISLAND',
   'NEAR BAY',
   'NEAR OCEAN']}}

In [ ]:
columns = train_data.columns
columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [ ]:
column_dtypes = train_data.dtypes
column_dtypes =  dict(column_dtypes)
str(column_dtypes["longitude"])

'float64'

In [ ]:
_column_dtypes = []
for column in columns:
    _column_dtypes.append(str(column_dtypes[column]))

In [ ]:
_column_dtypes

['float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'object']

In [ ]:
if len(schema_file_obj["columns"]) == len(columns):
    print("same number of columns")

same number of columns


In [ ]:
column_len = schema_file_obj["columns"].keys()
column_len = list(column_len)
for i in range(len(column_len)):
    if columns[i] not in column_len:
        print("something is wrong")

In [ ]:
_dtype = schema_file_obj["columns"]
_dtype

{'longitude': 'float',
 'latitude': 'float',
 'housing_median_age': 'float',
 'total_rooms': 'float',
 'total_bedrooms': 'float',
 'population': 'float',
 'households': 'float',
 'median_income': 'float',
 'median_house_value': 'float',
 'ocean_proximity': 'category'}

In [ ]:
chech_dtyepes= dict(zip(columns,_column_dtypes))

In [ ]:
chech_dtyepes

{'longitude': 'float64',
 'latitude': 'float64',
 'housing_median_age': 'float64',
 'total_rooms': 'float64',
 'total_bedrooms': 'float64',
 'population': 'float64',
 'households': 'float64',
 'median_income': 'float64',
 'median_house_value': 'float64',
 'ocean_proximity': 'object'}

In [ ]:
categorical_col = []
for i in columns:
    if chech_dtyepes[i] == "object":
        categorical_col.append(i)
        chech_dtyepes[i] = "category" 
    if _dtype[i] not in chech_dtyepes[i]:
        print("something's wrong")

In [ ]:
unique_val = {}
for value in categorical_col:
    unique_val[value] = list(train_data[value].unique())
unique_val

{'ocean_proximity': ['INLAND',
  'NEAR OCEAN',
  '<1H OCEAN',
  'NEAR BAY',
  'ISLAND']}

In [ ]:
values = schema_file_obj["domain_value"]
values

{'ocean_proximity': ['<1H OCEAN',
  'INLAND',
  'ISLAND',
  'NEAR BAY',
  'NEAR OCEAN']}

In [ ]:
for value in values:
    for check_values in values[value]:
        if check_values not in unique_val[value]:
            print(f"something's wrong in {value} column")

In [ ]:
numerical_col = []
for i in chech_dtyepes.keys():
    if "float" in chech_dtyepes[i] or "int" in chech_dtyepes[i]:
        numerical_col.append(i)
numerical_col

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

In [ ]:
obtained_obj = {"columns":chech_dtyepes,"numerical_col":numerical_col,"categorical_col":categorical_col,"domain_value":unique_val}
obtained_obj

{'columns': {'longitude': 'float64',
  'latitude': 'float64',
  'housing_median_age': 'float64',
  'total_rooms': 'float64',
  'total_bedrooms': 'float64',
  'population': 'float64',
  'households': 'float64',
  'median_income': 'float64',
  'median_house_value': 'float64',
  'ocean_proximity': 'category'},
 'numerical_col': ['longitude',
  'latitude',
  'housing_median_age',
  'total_rooms',
  'total_bedrooms',
  'population',
  'households',
  'median_income',
  'median_house_value'],
 'categorical_col': ['ocean_proximity'],
 'domain_value': {'ocean_proximity': ['INLAND',
   'NEAR OCEAN',
   '<1H OCEAN',
   'NEAR BAY',
   'ISLAND']}}

In [49]:
def get_obtain_obj(dataset):
    columns = dataset.columns
    column_dtypes = dict(dataset.dtypes)
    numerical_col = []
    categorical_col = []
    for i in column_dtypes:
        column_dtypes[i] = str(column_dtypes[i])
        if "float" in column_dtypes[i] or "int" in column_dtypes[i]:
            numerical_col.append(i)
        elif "object" in column_dtypes[i]:
            column_dtypes[i] = "category"
            categorical_col.append(i)
    domain_value = {}
    for value in categorical_col:
        domain_value[value] = list(dataset[value].unique())
    obtained_obj = {"columns":column_dtypes,"numerical_column":numerical_col,"categorical_column":categorical_col,"domain_value":domain_value}
    return obtained_obj

obtained_train_obj = get_obtain_obj(train_data)
obtained_test_obj = get_obtain_obj(test_data)

In [2]:
if True:
    l = []
    l.append("something different")
l = "something random"
l

'something random'

In [ ]:
obtained_train_obj["columns"]

{'longitude': 'float64',
 'latitude': 'float64',
 'housing_median_age': 'float64',
 'total_rooms': 'float64',
 'total_bedrooms': 'float64',
 'population': 'float64',
 'households': 'float64',
 'median_income': 'float64',
 'median_house_value': 'float64',
 'ocean_proximity': 'object'}

In [52]:
def compare_schema_to_dataset(schema_file_obj,obtained_obj,params):
    #checking the number of columns
    if len(schema_file_obj["columns"]) != len(obtained_obj["columns"]):
        return f"schema file and {params} data set do not have same number of columns"

    #checking the name of the columns and checking the data types of the column
    for val in schema_file_obj["columns"].keys():
        if val not in obtained_obj["columns"].keys():
            return f"{val} not present in {params} data"
        schema_type = schema_file_obj["columns"][val]
        obtain_type = obtained_obj["columns"][val]
        if schema_type not in obtain_type:
            return f"{val} is present in {params} data but data types {schema_type} do not match with {obtain_type}"
        if "float" in schema_type or "int" in schema_type:
            if val not in obtained_obj["numerical_column"]:
                return f"{val} is present in {params} data and data types {schema_type} also match with {obtain_type} but {val} is {schema_type} column and still not present in {obtained_obj['numerical_column']}"
        if "categorical" in schema_type:
            if val not in obtained_obj["categorical_column"]:
                return f"{val} is present in {params} data and data types {schema_type} also match with {obtain_type} but {val} is {schema_type} column and still not present in {obtained_obj['categorical_column']}"

    #to check domain value in categorical columns
    cat_col = schema_file_obj["categorical_columns"]
    domain_value = schema_file_obj["domain_value"]
    for value in cat_col:
        for d_value in domain_value[value]:
            if d_value not in obtained_obj["domain_value"][value]:
                return f"{d_value} is not described in {obtained_obj['domain_value'][value]} of {val} column of {params} data"

    return f"""The validation for {params} data that has been done are
    1. Same number of column in schema and {params} data
    2. The column names and data types are same in schema and {params} data
    3. The numerical and categorical column are same in both schema and {params} data
    4. The domain values are same for each categorical column in schema and {params} data
    """

    

def check_data_validation(schema_file_obj,obtained_train_obj,obtained_test_obj):
    #checking for number of columns
    print(compare_schema_to_dataset(schema_file_obj,obtained_train_obj,"train"))
    print(compare_schema_to_dataset(schema_file_obj,obtained_test_obj,"test")) 
check_data_validation(schema_file_obj,obtained_train_obj,obtained_test_obj)

The validation for train data that has been done are
    1. Same number of column in schema and train data
    2. The column names and data types are same in schema and train data
    3. The numerical and categorical column are same in both schema and train data
    4. The domain values are same for each categorical column in schema and train data
    
The validation for test data that has been done are
    1. Same number of column in schema and test data
    2. The column names and data types are same in schema and test data
    3. The numerical and categorical column are same in both schema and test data
    4. The domain values are same for each categorical column in schema and test data
    
